In [ ]:
import IPython.display
import svgwrite
import numpy as np
import os
import random
import pandas as pd
from random import randint
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
from torch.distributions.categorical import Categorical
from torch.distributions.normal import Normal
from torch.distributions.multivariate_normal import MultivariateNormal
import warnings
import cmath
from svgpathtools import *
import math
from deepsvg.svglib.svg import SVG

In [ ]:
import deepsvg.svglib.svg

In [ ]:
cuda0 = torch.device('cuda:0')

In [ ]:
paths, attributes, svg_attributes = svg2paths2('Minisample/leaves_base_2.svg')

In [ ]:
wsvg(paths[60],filename='guided_01.svg')
plant = SVG.load_svg('guided_01.svg').normalize().canonicalize() #.simplify_heuristic()
plant.save_svg('guided_01.svg')
new_path, _, _ = svg2paths2('guided_01.svg')
nodes = []
for segment in new_path[0]:
    nodes.append(segment.end)
wsvg(new_path,nodes=nodes,filename='guided_01_dots.svg') #
IPython.display.SVG('guided_01_dots.svg')

In [ ]:
path, attributes, svg_attributes = svg2paths2('guided_01.svg')
end_points = [path[0].start]
for segment in path[0]:
    end_points.append(segment.end)
end_points = np.array(end_points)
print(end_points[np.where(np.sqrt((end_points[1:]-end_points[:-1]).real**2 + (end_points[1:]-end_points[:-1]).imag**2)<=1)])
nodes = list(map(complex,end_points[np.where(np.sqrt((end_points[1:]-end_points[:-1]).real**2 + (end_points[1:]-end_points[:-1]).imag**2)<=1)]))
wsvg(new_path,nodes=nodes,filename='guided_02_dots.svg') #
IPython.display.SVG('guided_02_dots.svg')

In [ ]:
paths, attributes, svg_attributes = svg2paths2('Minisample/leaves_base_2.svg')

In [ ]:
len(paths)

In [ ]:
def perform_transformations(points):
    figures = [torch.unsqueeze(points,0)]
    #figures.append(torch.unsqueeze(transformed_tensor(points,flip_x_matrix),0))
    #figures.append(torch.unsqueeze(transformed_tensor(points,flip_y_matrix),0))
    #figures.append(torch.unsqueeze(transformed_tensor(points,flip_o_matrix),0))
    #figures.append(torch.unsqueeze(transformed_tensor(points,flip_xy_matrix),0))
    return figures

In [ ]:
n_seg = 584 #296#152 #produces equally sized vectors
ts = np.array(range(n_seg+1))*1/n_seg
inputs = []
figures = []
for i,path in enumerate(paths):
    wsvg(path,filename='variable_01.svg')
    plant = deepsvg.svglib.svg.SVG.load_svg('variable_01.svg').normalize().canonicalize()
    plant.save_svg('variable_01.svg')
    path, _, _ = svg2paths2('variable_01.svg')
    path = path[0]
    end_points = [path.start]
    for segment in path:
        end_points.append(segment.end)
    end_points = np.array(end_points)
        #print(end_points[np.where(np.sqrt((end_points[1:]-end_points[:-1]).real**2 + (end_points[1:]-end_points[:-1]).imag**2)<=0.5)])
    key_points = torch.tensor(list(map(complex,end_points[np.where(np.sqrt((end_points[1:]-end_points[:-1]).real**2 + (end_points[1:]-end_points[:-1]).imag**2)<=1)]))) #.float())
    if i==0 or i ==7:
        print(key_points)
    inputs.append(torch.unsqueeze(torch.cat((key_points,torch.zeros(10-len(key_points)))),0))
    points = torch.tensor([[path.point(t).real,path.point(t).imag] for t in ts]).float()
    figures += perform_transformations(points)
input_data = torch.cat(inputs).to(cuda0)
training_data = torch.cat(figures).to(cuda0) #dtype=torch.float,device=cuda0
training_data = training_data[:,1:,:] - training_data[:,:-1,:] #OPTIONAL
training_data = training_data.permute((0,2,1))
print(input_data.shape,training_data.shape)

In [ ]:
input_data_split = torch.cat((torch.unsqueeze(input_data.real.float(),1),torch.unsqueeze(input_data.imag.float(),1)),dim=1)

In [ ]:
input_data_split = input_data_split[:,:,1:]-input_data_split[:,:,:-1]

In [ ]:
input_data_split = input_data_split/torch.max(input_data_split)

In [ ]:
input_data_split.shape

In [ ]:
x_batches = torch.split(input_data_split,32)
y_batches = torch.split(training_data,32)

In [ ]:
class CNNArtist(nn.Module):

    def __init__(self): #, dropout=0.1
        super(CNNArtist, self).__init__()
    
        self.decoder = nn.Sequential(
            nn.ConvTranspose1d(2, 2, kernel_size=5,stride=1),
            nn.LeakyReLU(True),
            nn.ConvTranspose1d(2, 2, kernel_size=5,stride=2),
            nn.LeakyReLU(True),
            nn.ConvTranspose1d(2, 2, kernel_size=4,stride=2),
            nn.LeakyReLU(True),
            nn.ConvTranspose1d(2, 2, kernel_size=5,stride=3),
            nn.LeakyReLU(True),
            nn.ConvTranspose1d(2, 2, kernel_size=5,stride=3),
            nn.LeakyReLU(True),
            nn.Linear(584,584)) 
    def forward(self, x):
        return self.decoder(x)

In [ ]:
class Artist(nn.Module):

    def __init__(self): #, dropout=0.1
        super(CNNArtist, self).__init__()
    
        self.decoder = nn.Sequential(
            nn.ConvTranspose1d(2, 2, kernel_size=5,stride=1),
            nn.LeakyReLU(True),
            nn.ConvTranspose1d(2, 2, kernel_size=5,stride=2),
            nn.LeakyReLU(True),
            nn.ConvTranspose1d(2, 2, kernel_size=4,stride=2),
            nn.LeakyReLU(True),
            nn.ConvTranspose1d(2, 2, kernel_size=5,stride=3),
            nn.LeakyReLU(True),
            nn.ConvTranspose1d(2, 2, kernel_size=5,stride=3),
            nn.LeakyReLU(True),
            nn.Linear(584,584)) 
    def forward(self, x):
        return self.decoder(x)

In [ ]:
model = CNNArtist().to(cuda0)
mse_loss = nn.MSELoss()
#lr = 0.5 # learning rate
#optimizer = torch.optim.SGD(model.parameters(), lr=lr)
optimizer = torch.optim.Adam(model.parameters(),weight_decay=1e-5)
for epoch in range(4000):
    total_loss = 0
    for i,batch in enumerate(x_batches):
        optimizer.zero_grad()
        rec = model(batch)
        loss = mse_loss(rec,y_batches[i])
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1) #what param to set here? (grads are exploding without cliping)
        optimizer.step()
        total_loss += loss.item()
    if epoch % 100 == 0:
        print(total_loss) #with 10 epochs 7720 2.36    

In [ ]:
s = 6 #0.006
out = model(x_batches[0][s:s+1,:,:])
#out = y_batches[0][s:s+1,:,:]
real_values = torch.tensor([[0.0,0.0]],device=cuda0)
for i in range(1,out.shape[2]):
    real_values= torch.cat((real_values,torch.unsqueeze(real_values[-1]+out[0,:,i],0)),0)
print(real_values.shape)
segments =[]
for i in range(1,real_values.shape[0]):   
        segments.append(Line(complex(real_values[i-1,0],real_values[i-1,1]),complex(real_values[i,0],real_values[i,1])))
segments.append(Line(complex(real_values[i,0],real_values[i,1]),complex()))
wsvg(segments,filename='test_conv_rec.svg') #nodes=[complex(),complex(real_values[real_values.shape[0]-1,0],real_values[real_values.shape[0]-1,1])],
IPython.display.SVG('test_conv_rec.svg') 

In [ ]:
s = 60 #0.006
#out = model(x_batches[0][s:s+1,:,:])
real_values = input_data_split[s,:,:].permute((1,0))[:7,:]
print(real_values)
segments =[]
for i in range(1,real_values.shape[0]):   
        segments.append(Line(complex(real_values[i-1,0],real_values[i-1,1]),complex(real_values[i,0],real_values[i,1])))
#segments.append(Line(complex(real_values[i,0],real_values[i,1]),complex()))
print(segments)
nodes = []
for i in range(real_values.shape[0]):
    nodes.append(complex(real_values[i,0],real_values[i,1]))
wsvg(segments,nodes = nodes,filename='test_conv_rec_dots.svg') #nodes=[complex(),complex(real_values[real_values.shape[0]-1,0],real_values[real_values.shape[0]-1,1])],
IPython.display.SVG('test_conv_rec_dots.svg') 

In [ ]:
real_values